In [9]:
from charm.toolbox.pairinggroup import PairingGroup, ZR, G1, G2, GT, pair
from charm.toolbox.ABEnc import ABEnc
from abenc_tf22 import DIPE_ABE, Inner_Product
group = PairingGroup('SS512')
n = 5
dipe = DIPE_ABE(n, group)

In [10]:
attributes = Inner_Product(group)
vec_x,vec_y = attributes.gen_x_y(n,[1,2])
sum([x*y for x,y in zip(vec_x,vec_y)])

0

In [11]:
pp = dipe.setup()
pks, sks = dipe.auth_setup(pp)

In [12]:
GID = group.random(ZR)
M = group.random(GT)
GID, M

(386591351392750010173459929727371764902143625318,
 [767479785009742645424970485650687203496389826638134299490589039496722877205482595171773332336086523416150410176675546945536520920381101722668631373092625, 3115369115962284198178209299626158507084719207589685240390739641531175867117402093532454796035255738187259980233588200289454838943812079117128282000853488])

In [13]:
D0, D1, K = dipe.keygen(pp, sks, GID, vec_x)

In [14]:
C = dipe.encrypt(pp, pks, vec_y, M)

In [15]:
dipe.decrypt(D0, D1, K, C, vec_y)

[767479785009742645424970485650687203496389826638134299490589039496722877205482595171773332336086523416150410176675546945536520920381101722668631373092625, 3115369115962284198178209299626158507084719207589685240390739641531175867117402093532454796035255738187259980233588200289454838943812079117128282000853488]